<a href="https://colab.research.google.com/github/427Stanley427/Python/blob/main/G%C3%A9n%C3%A9rer_paire_BTC_balance_v%C3%A9rifi%C3%A9_bat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install bitcoin

In [ ]:
pip install requests

In [ ]:
import requests
import bitcoin

def generate_bitcoin_keypair_with_prefix(prefixes):
    while True:
        private_key = bitcoin.random_key()
        public_key_uncompressed = bitcoin.privtopub(private_key)

        # Générer une adresse P2PKH
        address_p2pkh = bitcoin.pubtoaddr(public_key_uncompressed)
        if address_p2pkh.startswith("1") and "1" in prefixes:
            address_type = "P2PKH"
            address = address_p2pkh

        # Générer une adresse P2SH
        elif address_p2pkh.startswith("3") and "3" in prefixes:
            address_type = "P2SH"
            address = address_p2pkh

        # Générer une adresse SegWit P2WPKH
        else:
            address_segwit = bitcoin.pubtosegwitaddr(public_key_uncompressed)
            if address_segwit.startswith("bc1") and "bc1" in prefixes:
                address_type = "SegWit P2WPKH"
                address = address_segwit
            else:
                continue

        print(f"Vérification de l'adresse ({address_type}): {address}")  # Affiche l'adresse vérifiée
        balance = get_balance(address)

        if balance > 0:
            return {
                "private_key_hex": private_key,
                "wif_private_key": wif_private_key,
                "address_type": address_type,
                "address": address,
                "balance": balance
            }

def get_balance(address):
    url = f"https://api.blockchair.com/bitcoin/dashboards/address/{address}"
    response = requests.get(url)
    data = response.json().get("data", {})

    if not data:
        return 0

    address_data = data.get(address, {})
    if not address_data:
        return 0

    balance_satoshis = address_data.get("address", {}).get("balance", 0)
    return balance_satoshis / 100_000_000  # Convertir en BTC

prefixes_to_check = ["1", "3", "bc1"]
kp = generate_bitcoin_keypair_with_prefix(prefixes_to_check)

print("\nAdresse avec balance positive trouvée!")
print("Clé privée hex :", kp["private_key_hex"])
print("Clé privée WIF :", kp["wif_private_key"])
print(f"Adresse ({kp['address_type']}) :", kp["address"], "| Balance:", kp["balance"], "BTC")
